In [7]:
import sys
sys.path.append('/home/monteirod/script/utils/')
import utils as u
import numpy as np
import xarray as xr
import geopandas
import matplotlib.pyplot as plt
import regionmask
import shapefile as shp
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import datetime
from scipy import stats
import sys
import dask
from dask.diagnostics import ProgressBar

# Resampled spatially, temporally and save datasets

In [8]:
### Homogenized in terms of spatial extent and averaged monthly the different datasets
### then save it in a repository
### The targeted format is dimension : time, spatial_dim / coordinates : lon, lat, ZS (orography) / variables : variable 

data_path = '/cnrm/cen/users/NO_SAVE/monteirod/' # Roots for the different repertory used
shp_path = '/cnrm/cen/users/NO_SAVE/monteirod/Shapefile/NUTS0_UE_ALPS' # Location of the used shapefile
shp = geopandas.read_file(shp_path) # Read shapefile with geopandas

## Boundaries of the domain to be subset : Alpine ridge extension
lat_inf, lat_sup = 43,49
lon_inf, lon_sup = 4,17

## Path of the outputs
out_reportory = '/cnrm/cen/users/NO_SAVE/monteirod/dataset_monthly/'

## Enable/disable SCD (snow cover duration) / SD (snow duration)
sd = False
scd = True
threshold_scd = 0.05 # Threshold used for snow depth (m) to determine whether there is snow or not

## List of the dataset on which the program will applied
ls_dataset = ['OBS_insitu','MTMSI','UERRA','AROME','ERA5','ERA5_CROCUS','ERA5_Land', 'CERRA_Land'] # full list
# ls_dataset = ['CERRA_Land']

## Datasets

In [10]:
for i in range(0,len(ls_dataset)):
    dataset = ls_dataset[i]
    var = 'snd' # Specify which variable, snd (snow depth), tas (temperature), pr (precipitation)
    print(dataset)

    ### Obs matiu ###
    
    if dataset == 'OBS_insitu' and var == 'snd':
        ds = xr.open_mfdataset(data_path+'Reference_Dataset/SND_OBS/OBS_Matiu/snd_Alps_1961_2020.nc', chunks = 'auto') # Open file
        
        # Pre-treatment
        ds = ds.drop(['HS_after_qc'])
        ds = u.sellonlatbox(ds,['Station_Name'],lat_inf,lat_sup,lon_inf,lon_sup, avoid_reindex=False) # Resampled spatially
        ds['snd'] = ds['snd']/100 # cm --> m
        
        if scd == True:
            var = 'scd'
            ds = ds['snd'].sel(time = slice('1985','2015')).dropna(dim = 'Station_Name').resample(time = 'A-SEP', closed = 'left',label = 'left').map(u.lcscd, threshold = threshold_scd)
        
        elif sd == True:
            var = 'sd'
            ds = ds['snd'].sel(time = slice('1985','2015')).dropna(dim = 'Station_Name').resample(time = '1MS', closed = 'left',label = 'left').map(u.lcscd, threshold = threshold_scd)
        
        else:
            ds = ds.resample(time = '1MS', closed = 'left',label = 'left').mean()

    ### LAPrec ###

    if dataset == 'LAPrec' and var == 'pr':
        ds = xr.open_mfdataset(data_path+'Reference_Dataset/LAprec/LAPrec1901.v1.1.nc', chunks = 'auto')

        # Pre-treatment
        ds = ds.drop('lambert_azimuthal_equal_area').set_coords(['lat','lon','dem']).rename({"LAPrec1901":"pr", 'dem':'ZS','X':'x','Y':'y'})  
        ds['pr'] = ds['pr']/ds.time.dt.days_in_month ## transform monthly pr sum to daily weight by number of day in month

        ds = u.sellonlatbox(ds,['x','y'],lat_inf,lat_sup,lon_inf,lon_sup)
        ds = ds.resample(time = '1MS', closed = 'left',label = 'left').mean()

    ### E-OBS tas ###

    if dataset == 'E_OBS' and var == 'tas':
        ds = xr.open_mfdataset(data_path+'Reference_Dataset/E-OBS_0.1degree/tas/monthly/tg_ens_mean_0.1deg_reg_v23.1e_monthly.nc', chunks = 'auto')

        # Pre-treatment
        ds_spread = xr.open_mfdataset(data_path+'Reference_Dataset/E-OBS_0.1degree/tas/monthly/tg_ens_spread_0.1deg_reg_v23.1e_monthly.nc', chunks = 'auto').rename({'tg':'tas_spread'})
        ds_ZS = xr.open_mfdataset(data_path+'Reference_Dataset/E-OBS_0.1degree/fx/*.nc')
        
        # Rounded issues between the orography files and the other, so we have to round it ourself
        ds['latitude'] = ds['latitude'].round(decimals = 5)
        ds['longitude'] = ds['longitude'].round(decimals = 5)
        ds_ZS['latitude'] = ds_ZS['latitude'].round(decimals = 5)
        ds_ZS['longitude'] = ds_ZS['longitude'].round(decimals = 5)
        ds_spread['latitude'] = ds_spread['latitude'].round(decimals = 5)
        ds_spread['longitude'] = ds_spread['longitude'].round(decimals = 5)
        
        ds = xr.merge([ds,ds_ZS, ds_spread])
        ds = ds.rename({"tg":"tas",'latitude':'lat','longitude':'lon', 'elevation':'ZS'}).drop('time_bnds').set_coords('ZS')

        ds = u.sellonlatbox(ds,['lat','lon'],lat_inf,lat_sup,lon_inf,lon_sup, avoid_reindex=False)
        ds = ds.resample(time = '1MS', closed = 'left',label = 'left').mean()
    
    ### E-OBS tas HOM ###
    
    if dataset == 'E_OBS_HOM' and var == 'tas':
        ds = xr.open_mfdataset(data_path+'Reference_Dataset/E-OBS_0.1degree/tas_homogenized/monthly/tg_ens_mean_monthly_0.1deg_reg_v19.0eHOM.nc', chunks = 'auto')

        # Pre-treatment
        ds_ZS = xr.open_mfdataset(data_path+'Reference_Dataset/E-OBS_0.1degree/fx/*.nc')
        
        # Rounded issues, so we have to round it ourself
        ds['latitude'] = ds['latitude'].round(decimals = 5)
        ds['longitude'] = ds['longitude'].round(decimals = 5)
        ds_ZS['latitude'] = ds_ZS['latitude'].round(decimals = 5)
        ds_ZS['longitude'] = ds_ZS['longitude'].round(decimals = 5)
        
        ds = xr.merge([ds,ds_ZS])
        ds = ds.rename({"tg":"tas",'latitude':'lat','longitude':'lon', 'elevation':'ZS'}).drop('time_bnds').set_coords('ZS')

        ds = u.sellonlatbox(ds,['lat','lon'],lat_inf,lat_sup,lon_inf,lon_sup, avoid_reindex=False)
        ds = ds.resample(time = '1MS', closed = 'left',label = 'left').mean()

    ### MTMSI ### 

    if dataset == 'MTMSI':
        
        if var == 'snd':
            ds = xr.open_mfdataset(data_path+'UERRA_CLIMTOUR_NN/tas_pr_snd/PRO_*.nc', chunks = 'auto').rename({'DSN_T_ISBA':'snd'}).drop(['latitude','longitude','ZS']) # Drop lat, lon, ZS because they contain false values

        elif var == 'tas' or var == "pr":
            ds = xr.open_mfdataset(data_path+'UERRA_CLIMTOUR_NN/tas_pr_snd/FORCING_UERRA_*.nc', chunks = 'auto').drop(['LAT','LON','ZS'])

        ds_ZS = xr.open_dataset(data_path+'UERRA_CLIMTOUR_NN/tas_pr_snd/FORCING_UERRA_198208010600_1983080106000_EUROPE.nc').isel(time = 0).squeeze()[['ZS', 'LAT','LON']] # Real lat, lon, ZS values
        ds = xr.merge([ds,ds_ZS])

        ## Pre-treatment
        ds = ds.rename({'LAT':'lat','LON':'lon'}).set_coords(['ZS','lat','lon'])
        ds = u.sellonlatbox(ds,['Number_of_points'],lat_inf,lat_sup,lon_inf,lon_sup)

        if scd == True:
            var = 'scd'
            ds = ds['snd'].resample(time = 'A-SEP', closed = 'left',label = 'left').map(u.lcscd, threshold = threshold_scd)
            
        elif sd == True:
            var = 'sd'
            ds = ds['snd'].resample(time = '1MS', closed = 'left',label = 'left').map(u.lcscd, threshold = threshold_scd)
        
        else:
            ds = ds.resample(time = '1MS', closed = 'left',label = 'left').mean()

        ## Post treatment 
        if var == 'tas':
            ds['tas'] = ds['Tair'] - 273.15
            ds = ds[['tas','ZS']]
            
        elif var == 'pr':
            ds['pr'] = ds['Rainf']*3600*24 + ds['Snowf']*3600*24
            ds = ds[['pr','ZS']]    

    ### UERRA ### 

    if dataset == 'UERRA':
        
        if var == 'tas' or var == 'pr':
            ds = xr.open_mfdataset(data_path+'UERRA_MESCAN_SURFEX/MESCAN-SURFEX_an/'+var+'/'+var+'_UERRA_MESCAN_SURFEX_day_*.nc', chunks = 'auto', combine = 'nested',concat_dim='time', coords = 'minimal', data_vars='minimal')
        
        elif var == 'snd':
            ds = xr.open_mfdataset(data_path+'UERRA_MESCAN_SURFEX/SURFEX_fc/'+var+'/'+var+'_UERRA_MESCAN_SURFEX_day_*.nc', chunks = 'auto', combine = 'nested',concat_dim='time', coords = 'minimal', data_vars='minimal')

        ## Pre-treatment
        if var == 'tas':
            ds = ds.drop(['height','time_bnds','LAT','LON']).squeeze() # Forcings are in SURFEX format, so we transform it

        ds_ZS = xr.open_dataset('/cnrm/cen/users/NO_SAVE/monteirod/UERRA_MESCAN_SURFEX/fx/ZS_lon_lat_UERRA_MESCAN.nc')
        ds = xr.merge([ds_ZS,ds]).rename({'LAT':'lat','LON':'lon'}).set_coords(['ZS','lat','lon'])
        ds = u.sellonlatbox(ds,['x','y'],lat_inf,lat_sup,lon_inf,lon_sup)

        if scd == True:
            var = 'scd'
            ds = ds['snd'].resample(time = 'A-SEP', closed = 'left',label = 'left').map(u.lcscd, threshold = threshold_scd)
            
        elif sd == True:
            var = 'sd'
            ds = ds['snd'].resample(time = '1MS', closed = 'left',label = 'left').map(u.lcscd, threshold = threshold_scd)
        
        else:
            ds = ds.resample(time = '1MS', closed = 'left',label = 'left').mean()

        ## Post-treatment 
        if var == 'tas':
            ds =  ds.rename({'2t':'tas'})
            ds['tas'] = ds['tas'] - 273.15
            ds = ds[['tas','ZS']]
        
        elif var == 'pr':
            ds =  ds.rename({'tp':'pr'})
            ds = ds[['pr','ZS']]

    ### ALADIN ### 

    if dataset == 'ALADIN':
        
        if var == 'tas' or var == 'pr':
            ds = xr.open_mfdataset(data_path+'ALADIN/ECMWF-ERAINT/tas_pr/*.nc', chunks = 'auto')
            
        elif var == 'snd' or var == 'snc':
            ds = xr.open_mfdataset(data_path+'ALADIN/ECMWF-ERAINT/ALADIN_SL2D/*.nc', chunks = 'auto')    

        ## Pre-treatment
        ds = ds.rename({'time_counter':'time','nav_lat':'lat','nav_lon':'lon'}).isel(x = slice(8,413),y = slice(8,269)) # rename and remove I zone
        ds_ZS = xr.open_dataset('/cnrm/cen/users/NO_SAVE/monteirod/ALADIN/fx/orog_MED-11_ECMWF-ERAINT_evaluation_r1i1p1_CNRM-ALADIN63_v1_fx.nc').rename({'orog':'ZS'}).drop(['lon_bnds','lat_bnds'])
        ds = u.sellonlatbox(ds,['x','y'],lat_inf,lat_sup,lon_inf,lon_sup)
        ds_ZS = u.sellonlatbox(ds_ZS,['x','y'],lat_inf,lat_sup,lon_inf,lon_sup)
        ds = xr.merge([ds_ZS,ds]).set_coords('ZS')
        ds = ds.resample(time ='1MS').mean(dim = 'time')

        ## Post-treatment
        if var == "tas":
            ds['tas'] = ds['tas'] - 273.15

        elif var == 'pr':
            ds['pr'] = ds['pr']*3600*24
        
        elif var == 'snc':
            ds['snc'] = ds['snc']*100

    ### AROME ###

    if dataset == 'AROME':
        
        data_path = '/cnrm/cen/users/NO_SAVE/monteirod/'
        ds = xr.open_mfdataset(data_path+'AROME/ECMWF-ERAINT/'+var+'/'+var+'_ALP-3_ECMWF-ERAINT_evaluation_r1i1p1_CNRM-AROME41t1_fpsconv-x1n2-v1_day_*.nc', chunks = 'auto')

        ## Pre-treatment
        ds_ZS = xr.open_dataset(data_path+'AROME/fx/orog_ALP-3_ECMWF-ERAINT_evaluation_r0i0p0_CNRM-AROME-41_fpsconv-x2yn2v1_fx.nc').rename({'X':'x','Y':'y', 'orog':'ZS', 'latitude':'lat','longitude':'lon'})
        
        # Reindex the same ways the orography file and main file
        ds_ZS['x'] = ds['x']
        ds_ZS['y'] = ds['y']

        ds = u.sellonlatbox(ds,['x','y'],lat_inf,lat_sup,lon_inf,lon_sup)
        ds_ZS = u.sellonlatbox(ds_ZS,['x','y'],lat_inf,lat_sup,lon_inf,lon_sup).drop(['lon','lat'])
        ds = xr.merge([ds,ds_ZS]).set_coords(['ZS','lat','lon'])

        if scd == True:
            var = 'scd'
            ds = ds['snd'].resample(time = 'A-SEP', closed = 'left',label = 'left').map(u.lcscd, threshold = 0.01)
            
        elif sd == True:
            var = 'sd'
            ds = ds['snd'].resample(time = '1MS', closed = 'left',label = 'left').map(u.lcscd, threshold = 0.01)
        
        else:
            ds = ds.resample(time = '1MS', closed = 'left',label = 'left').mean()

        if var == 'tas':
            ds['tas'] = ds['tas'] - 273.15

        if var == 'pr':
            ds['pr'] = ds['accpluie']*3600 + ds['accneige']*3600 + ds['graupel']*3600

            
    ### ERA5 ###

    if dataset == 'ERA5':

        ds = xr.open_mfdataset(data_path+"ERA5/"+var+'/*.nc', chunks = 'auto')

        ## Pre-treatment
        ds_ZS = (xr.open_dataset('/cnrm/cen/users/NO_SAVE/monteirod/ERA5/fx/ERA5_geopotential.nc').squeeze()/9.80665)
        ds_ZS = ds_ZS.rename({'z':'ZS','latitude':'lat','longitude':'lon'})
        
        if var == 'snd':
            ds = ds.sel(longitude = slice(4,17),latitude = slice(50,43))
            ds_ZS = ds_ZS.sel(lon = slice(4,17),lat = slice(50,43))
        
        ds = ds.rename({'latitude':'lat','longitude':'lon'})

        ds = u.sellonlatbox(ds,['lon','lat'],lat_inf,lat_sup,lon_inf,lon_sup)
        ds_ZS = u.sellonlatbox(ds_ZS,['lon','lat'],lat_inf,lat_sup,lon_inf,lon_sup).drop(['lon','lat'])
        ds = xr.merge([ds_ZS,ds]).set_coords(['ZS','lat','lon'])

        if var == 'tas':
            ds = ds.rename({'t2m':'tas'})
            ds['tas'] = ds['tas']-273.15
            
        elif var == 'pr':
            ds['pr'] = ds['tp']*1000
        
        elif var == 'snd':
            ds['snd'] = (ds['sd']*1000)/ds['rsn']
            ds = ds.drop(['sd','rsn'])

        if scd == True:
            var = 'scd'
            ds = ds['snd'].resample(time = 'A-SEP', closed='left', label='left').map(u.lcscd, threshold = threshold_scd)
            
        elif sd == True:
            var = 'sd'
            ds = ds['snd'].resample(time = '1MS', closed = 'left',label = 'left').map(u.lcscd, threshold = threshold_scd)
        
        else:
            ds = ds.resample(time = '1MS', closed = 'left',label = 'left').mean()
            

    ### ERA5-CROCUS ###

    if dataset == 'ERA5_CROCUS':

        data_path = '/cnrm/cen/users/NO_SAVE/monteirod/'

        ds = xr.open_mfdataset(data_path+'ERA5-CROCUS/'+var+'/'+var+'*.nc', chunks = 'auto')
        ds_ZS = xr.open_mfdataset(data_path+'ERA5-CROCUS/'+'fx/*.nc', chunks = 'auto')
        
        ## Pre-treatment
        ds['lat'] = ds_ZS['lat']
        ds['lon'] = ds_ZS['lon']

        ds = u.sellonlatbox(ds,['lon','lat'],lat_inf,lat_sup,lon_inf,lon_sup)
        ds_ZS = u.sellonlatbox(ds_ZS,['lon','lat'],lat_inf,lat_sup,lon_inf,lon_sup).drop(['lon','lat'])
        ds = xr.merge([ds_ZS,ds]).set_coords(['ZS','lat','lon'])

        if scd == True:
            var = 'scd'
            ds = ds['snd'].resample(time = 'A-SEP', closed = 'left',label = 'left').map(u.lcscd, threshold = threshold_scd)
            
        elif sd == True:
            var = 'sd'
            ds = ds['snd'].resample(time = '1MS', closed = 'left',label = 'left').map(u.lcscd, threshold = threshold_scd)
        
        else:
            ds = ds.resample(time = '1MS', closed = 'left',label = 'left').mean()

        if var == 'tas':
            ds = ds.drop('height')
            ds['tas']= ds['tas']-273.15
        
        elif var == 'pr':
            ds['pr']= ds['pr']*3600*24

    ### ERA5-Land ###

    if dataset == 'ERA5_Land':

        ds = xr.open_mfdataset(data_path+"ERA5-Land/"+var+'/*.nc', chunks = 'auto')
        ds_ZS = xr.open_dataset(data_path+"ERA5-Land/"+'fx/oro_ERA5-Land.nc').squeeze().rename({'z':'ZS','latitude':'lat','longitude':'lon'})/9.80665

        ds = u.sellonlatbox(ds,['lon','lat'],lat_inf,lat_sup,lon_inf,lon_sup, avoid_reindex = False)
        ds_ZS = u.sellonlatbox(ds_ZS,['lon','lat'],lat_inf,lat_sup,lon_inf,lon_sup, avoid_reindex = False).drop(['lon','lat'])
        ds = xr.merge([ds_ZS,ds]).set_coords(['ZS','lat','lon'])
        
        if var == 'tas':
            ds['tas'] = ds['t2m']-273.15
        elif var == 'pr':
            ds['pr'] = ds['tp']*1000
        
        elif var == 'snd':
            ds['snd'] = (ds['sd']*1000)/ds['rsn']
            ds = ds.drop(['sd','rsn'])            

        if scd == True:
            var = 'scd'
            ds = ds['snd'].resample(time = 'A-SEP', closed = 'left',label = 'left').map(u.lcscd, threshold = threshold_scd)
        
        elif sd == True:
            var = 'sd'
            ds = ds['snd'].resample(time = '1MS', closed = 'left',label = 'left').map(u.lcscd, threshold = threshold_scd)
        
        else:
            ds = ds.resample(time = '1MS', closed = 'left',label = 'left').mean()


    ### CERRA-Land ###

    if dataset == 'CERRA_Land':

        ds = xr.open_mfdataset(data_path+'CERRA-Land/'+var+'/*.nc', chunks = 'auto', combine = 'nested',concat_dim='time', coords = 'minimal', data_vars='minimal')
        ds_ZS = xr.open_dataset('/cnrm/cen/users/NO_SAVE/monteirod/UERRA_MESCAN_SURFEX/fx/ZS_lon_lat_UERRA_MESCAN.nc').rename({'LAT':'lat','LON':'lon'})
        ds = xr.merge([ds_ZS,ds]).set_coords(['lat','lon','ZS']).drop('Lambert_Conformal')
        ds = u.sellonlatbox(ds,['x','y'],lat_inf,lat_sup,lon_inf,lon_sup)

        if scd == True:
            var = 'scd'
            ds = ds['snd'].resample(time = 'A-SEP', closed = 'left',label = 'left').map(u.lcscd, threshold = threshold_scd)
        
        elif sd == True:
            var = 'sd'
            ds = ds['snd'].resample(time = '1MS', closed = 'left',label = 'left').map(u.lcscd, threshold = threshold_scd)
        
        else:
            ds = ds.resample(time = '1MS', closed = 'left',label = 'left').mean()

        if var == 'tas':
            ds['tas'] = ds['tas'] - 273.15
            ds = ds.squeeze().drop('height')
            
    # Prepare the job to be written in memory
    write_job = ds.to_netcdf(out_reportory+var+'/'+var+'5cm_'+dataset+'.nc', compute=False)

    # Write the job with a progress bar view
    with ProgressBar():
        print(dataset+var)
        write_job.compute()

OBS_insitu
OBS_insituscd
[########################################] | 100% Completed | 951.93 ms
MTMSI
MTMSIscd
[########################################] | 100% Completed | 1.17 sms
UERRA
UERRAscd
[########################################] | 100% Completed | 157.85 s
AROME
AROMEscd
[########################################] | 100% Completed | 192.16 s
ERA5
ERA5scd
[########################################] | 100% Completed | 6.80 ss
ERA5_CROCUS
ERA5_CROCUSscd
[########################################] | 100% Completed | 243.19 s
ERA5_Land
ERA5_Landscd
[########################################] | 100% Completed | 19.10 s
CERRA_Land
CERRA_Landscd
[########################################] | 100% Completed | 65.15 s


In [6]:
## MODIS specific case of computation 
from glob import glob

dataset = 'MODIS'
ls_year = np.arange(2000, 2010, 1) # Temporal extension of MODIS dataset

modis = []

for i in range(0, len(ls_year)): # Loop over the MODIS yearly files and merge them into one
    i_ds = xr.open_dataset(glob(data_path+'Reference_Dataset/MODIS_NDSI_500m_Simon/MODIS_NDSI_500m_'+
                           str(ls_year[i])+'*.nc')[0]).sel(time=slice('2000-03', '2009')).rename({'x':'lon','y':'lat'})

    # Some MODIS file have rounded issues so we homogenized everything
    i_ds['lat'] = np.round(i_ds['lat'], decimals=3)
    i_ds['lon'] = np.round(i_ds['lon'], decimals=3)
    
    modis.append(i_ds)
ds = xr.concat(modis, dim='time')

ds = ds.squeeze().rename({'Band1': 'snc'}) # Only keep NDSI fields

# Load orography files from a DEM at MODIS resolutions and cut it to match MODIS images spatial extension
oro = xr.open_dataset(data_path+'/Reference_Dataset/MODIS_NDSI_500m_Simon/MNT500m.nc').sel(lat=slice(43.5,
                                                                                                     48.5), lon=slice(5, 16.5)).rename({'Band1': 'ZS'})
oro['lat'] = ds['lat']
oro['lon'] = ds['lon']
ds = xr.merge([ds, oro]).set_coords('ZS')

if scd:
    var = 'scd'
    ds = ds['snc'].resample(time='A-SEP').map(u.lcscd, threshold=20)

elif sd:
    var = 'sd'
    ds = ds['snc'].resample(time='1MS').map(u.lcscd, threshold=20)

# Spatial subsampled
ds = u.sellonlatbox(ds, ['lat', 'lon'], lat_inf, lat_sup, lon_inf, lon_sup)

ds.to_netcdf(out_reportory+var+'/'+var+'_'+dataset+'.nc')

ValueError: cannot rename 'x' because it is not a variable or dimension in this dataset

## MODIS computation : NDSI from NSIDC

In [6]:
## MODIS specific case of computation 

dataset = 'MODIS_NSIDC'
var = 'scd'
data_path = '/cnrm/cen/users/NO_SAVE/monteirod/' # Roots for the different repertory used

## Path of the outputs
out_reportory = '/mnt/lfs/d10/mrns/users/NO_SAVE/monteirod/Reference_Dataset/MOD10A1_NSIDC/netCDF/scd/'

## Temporal extension of MODIS dataset
ls_year = np.arange(2000, 2015, 1)

# Try to open all at the time with dask
#i_ds = xr.open_mfdataset(data_path+'Reference_Dataset/MOD10A1_NSIDC/netCDF/MOD10A1F_CGF_NSIDC_*.nc', chunks={'x': 100, 'y': 100}, combine = 'nested',concat_dim='time', coords = 'minimal', data_vars='minimal').drop('band')

for i in range(0, len(ls_year)): # Loop over the MODIS yearly files and merge them into one
    print(i)
    i_ds = xr.concat([xr.open_mfdataset(glob(data_path+'Reference_Dataset/MOD10A1_NSIDC/netCDF/MOD10A1F_CGF_NSIDC_'+
                           str(ls_year[i])+'*.nc')[0], chunks = 'auto').rename({'x':'lon','y':'lat'}),xr.open_mfdataset(glob(data_path+'Reference_Dataset/MOD10A1_NSIDC/netCDF/MOD10A1F_CGF_NSIDC_'+
                           str(ls_year[i]+1)+'*.nc')[0], chunks = 'auto').rename({'x':'lon','y':'lat'})], dim = 'time').drop('band')

    # Some MODIS file have rounded issues so we homogenized everything
    
    if var == 'scd':
        var = 'scd'
        i_ds = i_ds['NDSI'].resample(time='A-SEP', closed = 'left', label = 'left').map(u.lcscd, threshold=20)

    elif var == 'sd':
        var = 'sd'
        i_ds = i_ds['NDSI'].resample(time='1MS', closed = 'left', label = 'left').map(u.lcscd, threshold=20)

    # Spatial subsampled
    # Load orography files from a DEM at MODIS resolutions 
    oro = xr.open_dataset(data_path+'/Reference_Dataset/MOD10A1_NSIDC/fx/DEM_MODIS_NSIDC.nc').rename({'Band1': 'ZS'})
    oro['lat'] = list(reversed(i_ds['lat']))
    oro['lon'] = i_ds['lon']
    i_ds = xr.merge([i_ds, oro]).set_coords('ZS')
    
    i_ds = i_ds.isel(time = 1) # to only select the timestep where you have a full year

    write_job = i_ds.to_netcdf(out_reportory+var+'_'+dataset+'_'+str(ls_year[i])+'.nc', compute=False)

    # Write the job with a progress bar view
    with ProgressBar():
        print(dataset+var)
        write_job.compute()
        
## Concat all yearly files of scd into one

var = 'scd'
dataset = 'MODIS'

ls_year = np.arange(2000, 2015, 1)
ds = []
for i in range(0,len(ls_year)):
    ds.append(xr.load_dataset(out_reportory+'scd_MODIS_NSIDC_'+str(ls_year[i])+'.nc'))
ds = xr.concat(ds, dim = 'time')

out_reportory = '/cnrm/cen/users/NO_SAVE/monteirod/dataset_monthly/'

ds.to_netcdf(out_reportory_repertory+var+'_'+dataset+'_NSIDC.nc')

0
MODIS_NSIDCscd
[########################################] | 100% Completed | 16.73 s
1
MODIS_NSIDCscd
[########################################] | 100% Completed | 15.71 s


# Shapefile production

In [ ]:
from shapely.ops import cascaded_union
from shapely.geometry import Polygon

#######################
## Alpine shp area  ###
#######################

## HISTALP intersect ALPCONV ## # Intersect the HISTALP division with the shapefile of the Alps (as defined by the Alpine convention)
HISTALP = geopandas.read_file('/cnrm/cen/users/NO_SAVE/monteirod/Shapefile/HISTALP_region/')
alp_conv = geopandas.read_file('/cnrm/cen/users/NO_SAVE/monteirod/Shapefile/Alpine_Convention_Perimeter_2018_v2_4326/')

HISTALP_Alps = geopandas.overlay(HISTALP, alp_conv, how='intersection') # Intersect both shapefile
NAME = ['SW', 'NW', 'NE', 'SE'] # Labelled the created regions 
HISTALP_Alps['NAME'] = NAME
HISTALP_Alps.to_file('/cnrm/cen/users/NO_SAVE/monteirod/Shapefile/HISTALP_Alps', encoding='UTF-8') # Save it

## NUTS0 intersect ALPCONV ## # Intersection of the NUTS0 division with Alpconv
NUTS = geopandas.read_file('/cnrm/cen/users/NO_SAVE/monteirod/Shapefile/NUTS/NUTS_RG_20M_2021_4326.shp')
NUTS0 = NUTS[NUTS['LEVL_CODE'].isin([0])]
alp_conv = geopandas.read_file('/cnrm/cen/users/NO_SAVE/monteirod/Shapefile/Alpine_Convention_Perimeter_2018_v2_4326/')
NUTS0_UE_ALPS = NUTS0.overlay(alp_conv, how='intersection')

# Select Swiss (CH) and Lichtenstein (LI) row from the intersect shapefile
CH = NUTS0_UE_ALPS.iloc[3:4]
LI = NUTS0_UE_ALPS.iloc[7:9]

# Create one row based on the two polygons
CHLI = CH.overlay(LI, how='union').dissolve()

# Removed Li and Hu country from our geodataframe #
NUTS0_UE_ALPS = NUTS0_UE_ALPS[NUTS0_UE_ALPS['NUTS_ID'] != 'LI']
NUTS0_UE_ALPS = NUTS0_UE_ALPS[NUTS0_UE_ALPS['NUTS_ID'] != 'HU']

# Replace the geometry of CH by our new geometry that includes LI inside CH
NUTS0_UE_ALPS['geometry'][3] = CHLI['geometry'][0]

NUTS0_UE_ALPS.to_file('/cnrm/cen/users/NO_SAVE/monteirod/Shapefile/NUTS0_UE_ALPS', encoding='UTF-8')

#######################
## Mountainous area ###
#######################

# Load the shapefile that gathered all european mountain regions 
shp = geopandas.read_file('/cnrm/cen/users/NO_SAVE/monteirod/Shapefile/EuropeanMountainAreas/')

## Create shapefile for the specific regions and only keep the largest regions

# For Pyrenees
Pyrenees = shp[shp['name_mm'] == 'Pyrenees']
Pyrenees = Pyrenees.where(Pyrenees['area_km2'] > 800).dropna()

# For the Carpathian
Carpates = shp[shp['name_mm'] == 'Carpathians']
Carpates = Carpates.where(Carpates['area_km2'] > 800).dropna()

# For Scandinavia
Scandinavia = shp[shp['name_mm'] == 'Nordic mountains']

# Create square surrounding Scandinavia because Nordic mountains include icelands that we want to exclude from the region
polygon = Polygon([(4*10**6, 3.75*10**6), (4*10**6, 5.50*10**6), (5.50*10**6,
                  5.50*10**6), (5.50*10**6, 3*10**6), (4*10**6, 3.75*10**6)])
poly_gdf = geopandas.GeoDataFrame([1], geometry=[polygon])

# Clip the square over Scandinavie shp, meaning that you only keep polygon inside your square
Scandinavia = Scandinavia.clip(poly_gdf)
Scandinavia = Scandinavia.where(Scandinavia['area_km2'] > 800).dropna()

# # Resume all polygons for a region into one
Pyrenees = Pyrenees.to_crs("EPSG:4326")
Carpates = Carpates.to_crs("EPSG:4326")
Scandinavia = Scandinavia.to_crs("EPSG:4326")
Pyrenees.dissolve().to_file('/cnrm/cen/users/NO_SAVE/monteirod/Shapefile/Pyrenees_shp', encoding='UTF-8')
Carpates.dissolve().to_file('/cnrm/cen/users/NO_SAVE/monteirod/Shapefile/Carpates_shp', encoding='UTF-8')
Scandinavia.dissolve().to_file('/cnrm/cen/users/NO_SAVE/monteirod/Shapefile/Scandinavia_shp', encoding='UTF-8')